The **Lagrangian** $ L(\phi, \dot{\phi}, t) $ describes the motion of a system, where:
- $ \phi $ = Vertical angle of Pendulum from support
- $ \dot{\phi} $ = Angular velocity of Pendulum from support
- $t $ = Time variable

The Euler-Lagrange equation is:
$$
\frac{d}{dt} \frac{\partial L}{\partial \dot{\phi}_j} = \frac{\partial L}{\partial \phi_j}
$$
This can be written as:
$$
\frac{d}{dt} \nabla_{\dot{\phi}} L = \nabla_\phi L
$$
where:
- $ \nabla_\phi L $ is the gradient w.r.t. $ \phi $
- $ \nabla_{\dot{\phi}} L $ is the gradient w.r.t. $ \dot{\phi} $

To take the time derivative of $ \nabla_{\dot{\phi}} L $, use the chain rule:
$$
\frac{d}{dt} \nabla_{\dot{\phi}} L = \frac{\partial}{\partial t} \nabla_{\dot{\phi}} L + \nabla^2_{\dot{\phi} \dot{\phi}} L \, \ddot{\phi} + \nabla^2_{\phi \dot{\phi}} L \, \dot{\phi}
$$
where:
- $ \nabla^2_{\dot{\phi} \dot{\phi}} L $ is the **Hessian w.r.t. $ \dot{\phi} $** (an $ n \times n $ matrix)
- $ \nabla^2_{\phi \dot{\phi}} L $ is the mixed partial derivative matrix

This reflects that $ L $ can depend arbitrarily on both $ \phi $ and $ \dot{\phi} $.

You want to solve for $ \ddot{\phi} $:
$$
\frac{d}{dt} \nabla_{\dot{\phi}} L = \nabla_\phi L
$$
Plug in the expanded chain rule:
$$
\frac{\partial}{\partial t} \nabla_{\dot{\phi}} L + \nabla^2_{\dot{\phi} \dot{\phi}} L \, \ddot{\phi} + \nabla^2_{\phi \dot{\phi}} L \, \dot{\phi} = \nabla_\phi L
$$
Rearrange:
$$
\nabla^2_{\dot{\phi} \dot{\phi}} L \, \ddot{\phi} = \nabla_\phi L - \nabla^2_{\phi \dot{\phi}} L \, \dot{\phi} - \frac{\partial}{\partial t} \nabla_{\dot{\phi}} L
$$
Multiply both sides by the inverse (if it exists) of the Hessian $ \nabla^2_{\dot{\phi} \dot{\phi}} L $:
$$
\ddot{\phi} = [\nabla^2_{\dot{\phi} \dot{\phi}} L]^{-1} [\nabla_\phi L - \nabla^2_{\phi \dot{\phi}} L \, \dot{\phi} - \frac{\partial}{\partial t} \nabla_{\dot{\phi}} L]
$$



In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class LNN_SoftPlus(nn.Module):
    def __init__(self):
        super(LNN_SoftPlus, self).__init__()
        self.Linear_Stack = nn.Sequential(
            nn.Linear(3, 128),
            nn.Softplus(),
            nn.Linear(128, 128),
            nn.Softplus(),
            nn.Linear(128, 128),
            nn.Softplus(),
            nn.Linear(128, 1)
        )

    def forward(self, phi, phi_dot, gamma , t):
        state = torch.cat([phi, phi_dot, t], dim=-1)
        return self.Linear_Stack(state).squeeze(-1)


In [ ]:

def Euler_Lagrange(phi, phi_dot, t, L_fn):
    phi.requires_grad(True)
    phi_dot.requires_grad(True)
    t.requires_grad(True)
    L = L_fn(phi, phi_dot, t)
    grad_phi = torch.autograd.grad(L, phi, create_graph= True, retain_graph=True)
    grad_phi_dot = torch.autograd.grad(L, phi_dot,  create_graph= True, retain_graph=True)

    Hess = torch.autograd.functional.hessian(
        lambda pd: L_fn(phi, pd, t), phi_dot,  create_graph= True)
    Jaco = torch.autograd.functional.jacobian(
        lambda p: torch.autograd.grad(L_fn(p, phi_dot, t), phi_dot, create_graph= True, retain_graph= True),
        phi, create_graph=True)

    TimeJac = torch.autograd.functional.jacobian(
        lambda tt: torch.autograd.grad(L_fn(phi, phi_dot, tt), phi_dot, create_graph=True, retain_graph=True),
        t, create_graph=True
    )

    if TimeJac.shape[-1] == 1:
        TimeJac = TimeJac.squeeze(-1)

    phi_ddot = torch.linalg.solve(Hess, grad_phi - Jaco @ phi_dot - TimeJac)
    return phi_ddot


In [ ]:
def rk4(func, state, time, dt):
    k1 = func(state, time)
    k2 = func(state + 0.5 * dt * k1, state + 0.5 * dt)
    k3 = func(state + 0.5 * dt * k2, state + 0.5 * dt)
    k4 = func(state + dt * k3, state + dt)
    return y + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)